In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from networks import NoKafnet, Kafnet
import utils.datasetsUtils.CIFAR as CIFAR
from utils.datasetsUtils.taskManager import SingleTargetClassificationTask, NoTask
from configs.configs import DefaultConfig
from torchvision.transforms import transforms
import torch
from Trainer import Trainer
import matplotlib.pyplot as plt
from collections import defaultdict
import copy
import numpy as np

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

True
1
GeForce GTX 1050


In [4]:
config = DefaultConfig()
print(config)

confing_no_ewt = copy.copy(config)
confing_no_ewt.USE_EWC = False
print(confing_no_ewt)

CONFIG PARAMETERS
BATCH_SIZE: 64
DEVICE: cuda
EPOCHS: 2
EWC_IMPORTANCE: 1000
EWC_SAMPLE_SIZE: 250
IS_CONVOLUTIONAL: True
ITERS: 1
L1_REG: 0.0001
LOSS: cross_entropy
LR: 0.001
MODEL_NAME: 
OPTIMIZER: SGD
RUN_NAME: default
USE_EWC: True
USE_TENSORBOARD: True

CONFIG PARAMETERS
BATCH_SIZE: 64
DEVICE: cuda
EPOCHS: 2
EWC_IMPORTANCE: 1000
EWC_SAMPLE_SIZE: 250
IS_CONVOLUTIONAL: True
ITERS: 1
L1_REG: 0.0001
LOSS: cross_entropy
LR: 0.001
MODEL_NAME: 
OPTIMIZER: SGD
RUN_NAME: default
USE_EWC: False
USE_TENSORBOARD: True



In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
)

dataset = CIFAR.Cifar10('../data/cifar10', SingleTargetClassificationTask(), download=True,
                        force_download=False, train_split=0.8, transform=transform, target_transform=None)
dataset.load_dataset()

dataset_noewt = CIFAR.Cifar10('../data/cifar10', SingleTargetClassificationTask(), download=True,
                        force_download=False, train_split=0.8, transform=transform, target_transform=None)

dataset_noewt.load_dataset()

../data/cifar10/download
task #0 with train 48000 and test 12000 images (label: airplane)
task #1 with train 48000 and test 12000 images (label: automobile)
task #2 with train 48000 and test 12000 images (label: bird)
task #3 with train 48000 and test 12000 images (label: cat)
task #4 with train 48000 and test 12000 images (label: deer)
task #5 with train 48000 and test 12000 images (label: dog)
task #6 with train 48000 and test 12000 images (label: frog)
task #7 with train 48000 and test 12000 images (label: horse)
task #8 with train 48000 and test 12000 images (label: ship)
task #9 with train 48000 and test 12000 images (label: truck)
../data/cifar10/download
task #0 with train 48000 and test 12000 images (label: airplane)
task #1 with train 48000 and test 12000 images (label: automobile)
task #2 with train 48000 and test 12000 images (label: bird)
task #3 with train 48000 and test 12000 images (label: cat)
task #4 with train 48000 and test 12000 images (label: deer)
task #5 with tra

In [ ]:
net_ewt = NoKafnet.CNN(dataset.tasks_number)
net_no_ewt = NoKafnet.CNN(dataset.tasks_number)

print('Numero di parametri rete classica: ', sum([torch.numel(p) for p in net_ewt.parameters()]))
#print('Numero di parametri KAFNET: ', sum([torch.numel(p) for p in kafnet.parameters()]))

Numero di parametri rete classica:  171732


In [ ]:
trainer_ewt = Trainer(net_ewt, dataset, config)
trainer_no_ewt = Trainer(net_no_ewt, dataset_noewt, confing_no_ewt)

In [ ]:
results_ewt = trainer_ewt.all_tasks()

Training task 0, epoch 1: 100%|██████████| 750/750 [00:18<00:00, 39.57it/s, loss=0.626, batch#=750]
Testing task 0: 188it [00:02, 70.54it/s, batch#=188]                         
Training task 0, epoch 2:  98%|█████████▊| 734/750 [00:16<00:00, 43.97it/s, loss=0.59, batch#=734] 

In [ ]:
results_no_ewt = trainer_no_ewt.all_tasks()

In [ ]:
n_task = len(results_no_ewt)
tot_epochs = max([len(i['metrics']) for _, i in results_no_ewt.items()])

fig = plt.figure(figsize=(12, 24))
ax = None

#ax = fig.add_subplot(1, 1, 1)

for i, task in enumerate(results_no_ewt.keys()):
    
    ewt = results_ewt[task]
    no_ewt = results_no_ewt[task]

    x = range(tot_epochs-len(ewt['metrics']) , tot_epochs)
    
    m_ewt = defaultdict(list)
    m_no_ewt = defaultdict(list)
    
    for item in ewt['metrics']:
        for k, v in item.items():
            m_ewt[k].append(v)

    for item in no_ewt['metrics']:
        for k, v in item.items():
            m_no_ewt[k].append(v)
            
    ax = fig.add_subplot(n_task, 1, i+1, sharex=ax) 
 
    acc = m['accuracy']
    
    ax.plot(x, m_ewt['accuracy'], label='ewt')
    ax.plot(x, m_no_ewt['accuracy'], label='no ewt')
    ax.set_xticks(range(tot_epochs),minor=False)
    ax.set_title("Task {}".format(task))
    ax.legend(loc="lower left")
    
fig.subplots_adjust(hspace=0)